# Задание:
1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.
4. Преобразуйте категориальные переменные с помощью pd.get_dummies(). 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 
7. Заполните пропуски в результатах.

Загрузим данные, проверим типы, пропущенные значения. 

In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin

In [12]:
cars = pd.read_csv('/Users/pavel/Desktop/karpov/7 СТАТИСТИКА+/cars.csv')
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [13]:
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используем столбец CarName, разбьем значения ячеек по пробелу и запишем в колонку первый элемент. Например:
```
'audi 100 ls' → 'audi'
```

In [14]:
cars['company'] = cars.CarName.apply(lambda x : x.split(' ')[0])

In [15]:
cars.drop('CarName', axis = 1, inplace = True)

Часть данных оказалась с ошибками. Исправим это
```
'maxda' → 'mazda'
'Nissan' → 'nissan'
'porcshce' → 'porsche'
'toyouta' → 'toyota'
'vokswagen' & 'vw' → 'volkswagen'
```

In [16]:
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [17]:
fix_dict = {'maxda' : 'mazda',
            'Nissan' : 'nissan',
            'porcshce' : 'porsche',
            'toyouta' : 'toyota',
            'vokswagen': 'volkswagen',
            'vw' : 'volkswagen'}
def company_fix(x, dic):
    if x in dic:
        s = x.replace(x, dic[x])
        return s
    return x
cars['company'] = cars.company.apply(lambda x: company_fix(x, fix_dict))
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [18]:
cars.company.nunique()

22

Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:
```
'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
```

После этого посчитаем корреляцию между price и другими переменными.

In [19]:
df = cars[['price','company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower']]
df

,price,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114
201,19045.0,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160
202,21485.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134
203,22470.0,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106


In [20]:
digital_columns = (cars[['price', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower']])
digital_columns.corr().round(2)

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.00,0.58,0.68,0.76,0.84,0.87,0.55,0.81
wheelbase,0.58,1.00,0.87,0.80,0.78,0.57,0.49,0.35
carlength,0.68,0.87,1.00,0.84,0.88,0.68,0.61,0.55
carwidth,0.76,0.80,0.84,1.00,0.87,0.74,0.56,0.64
curbweight,0.84,0.78,0.88,0.87,1.00,0.85,0.65,0.75
enginesize,0.87,0.57,0.68,0.74,0.85,1.00,0.58,0.81
boreratio,0.55,0.49,0.61,0.56,0.65,0.58,1.00,0.57
horsepower,0.81,0.35,0.55,0.64,0.75,0.81,0.57,1.00


In [21]:
df[['price', 'horsepower']].corr().round(2)

,price,horsepower
price,1.00,0.81
horsepower,0.81,1.00


Преобразуем категориальные переменные в dummies переменные

In [22]:
column_list = ['price', 'company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'wheelbase', 'carlength' ,'carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio', 'horsepower']
cars[column_list].dtypes

price             float64
company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
dtype: object

In [23]:
object_columns = ['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber']
digital_columns = (['price', 'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'boreratio', 'horsepower'])

In [24]:
df_dummy = pd.get_dummies(data=cars[object_columns], drop_first = True)

In [25]:
total_df = pd.concat((cars[digital_columns], df_dummy), axis=1)
total_df.head()

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,13495.0,88.6,168.8,64.1,2548,130,3.47,111,False,False,...,False,False,False,False,False,True,False,False,False,False
1,16500.0,88.6,168.8,64.1,2548,130,3.47,111,False,False,...,False,False,False,False,False,True,False,False,False,False
2,16500.0,94.5,171.2,65.5,2823,152,2.68,154,False,False,...,False,False,True,False,False,False,True,False,False,False
3,13950.0,99.8,176.6,66.2,2337,109,3.19,102,True,False,...,True,False,False,False,False,True,False,False,False,False
4,17450.0,99.4,176.6,66.4,2824,136,3.19,115,True,False,...,True,False,False,False,True,False,False,False,False,False


Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

In [26]:
pingouin.linear_regression(total_df.horsepower, total_df.price)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-3721.761494,929.849242,-4.002543,8.782473e-05,0.653088,0.651379,-5555.162771,-1888.360218
1,horsepower,163.263061,8.351479,19.549000,1.483437e-48,0.653088,0.651379,146.796293,179.729829


In [27]:
model = smf.ols(formula='price ~ horsepower', data=total_df).fit().summary()
print(model)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sat, 09 Dec 2023   Prob (F-statistic):           1.48e-48
Time:                        20:19:18   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

Теперь – две модели:

* модель со всеми предикторами
* модель со всеми предикторами, кроме марок машин

Обратим внимание на изменения в R^2 , коэффициентах и их значимости. Поэтому остамяем модель ,где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима**